# Import

In [1]:
!pip install tensorflow_hub

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import os
import re
import json
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import string
import tensorflow as tf
import tensorflow_hub as hub
import keras
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K



Using TensorFlow backend.


In [3]:
def construct_vocab(letters):
    unigram=list(letters)
    bigram=  [i+j for i in unigram for j in unigram]
    trigram= [i+j+m for i in unigram for j in unigram for m in unigram]
    print(len(trigram))
    vocab_list=unigram+bigram+trigram
    vocab= {v:i for i,v in enumerate(vocab_list)}
    return vocab

def cosine_similarity(e,v, divide=True):
    """
    #Input:
    #e = nxd input matrix with n row-vectors of dimensionality d (n is number of dictionary_keys)
    #v = mxd input matrix with m row-vectors of dimensionality d (m is number of test samples)
    # Output:
    # Matrix D of size nxm
    # s(i,j) is the cosinesimiarlity of embed(i,:) and test(j,:)
    """
    s=e.dot(v.T)
    if divide:
        b=np.expand_dims(np.linalg.norm(e,axis=1),1)+1e-16  # plus this small value to avoid division zero.
        a=np.expand_dims(np.linalg.norm(v,axis=1),1)+1e-16  # plus this small value to avoid division zero.
        s=np.divide(s,np.multiply(b,a.T))
    # ... until here
    return s


def findknn(D,k):
    """
    # D=cos_distance matrix
    # k = number of nearest neighbors to be found


    # Output:
    # indices = kxm matrix, where indices(i,j) is the i^th nearest neighbor of xTe(j,:)
    # dists = Euclidean distances to the respective nearest neighbors
    """
    k=min(k,D.shape[1])
    m = D.shape[0]
    ind = np.argsort(D, axis=1)

    indices = ind[:,::-1][:,:k]
   # print(indices)
    r = np.array([_ for _ in range(m)], dtype=np.int)
    r = np.array([r] * k).T   
    scores = D[r,indices] 
    #scores=np.sort(scores,axis=1)[:,::-1]
    return indices,scores
        
    

# Load Dataset

## vendor name mapping

In [4]:
Meta_file='question-python-data-science-project-mwsr7tgbeo-mapping_challenge.xlsx'
meta_df=pd.read_excel(Meta_file,sheetname='canonical_line_item_table')

for col in meta_df.columns:
    meta_df[col]=meta_df[col].apply(str).apply(str.strip)
    
label_map_df= meta_df.groupby('canonical_vendor_name')['canonical_line_item_name'].apply(lambda x: x.tolist())
label_map_df.head()

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


canonical_vendor_name
10 Minute Ventures                                      [Management Services]
ACORD                                                 [eForms Redistribution]
APCIA                                            [Net Associate - Annual Fee]
AWS                         [Amazon CloudFront, Amazon EC2 Container Regis...
Acqcom Digital Marketing                                      [Web Media Fee]
Name: canonical_line_item_name, dtype: object

In [5]:
label_map_dict=label_map_df.to_dict()

vendor_list=(label_map_dict.keys())
label_list=meta_df.canonical_line_item_name.to_list()
label_map_dict

{'10 Minute Ventures': ['Management Services'],
 'ACORD': ['eForms Redistribution'],
 'APCIA': ['Net Associate - Annual Fee'],
 'AWS': ['Amazon CloudFront',
  'Amazon EC2 Container Registry (ECR)',
  'Amazon EC2 Container Service',
  'Amazon Elastic Compute Cloud',
  'Amazon Elasticsearch Service',
  'Amazon Kinesis Firehose',
  'Amazon Lightsail',
  'Amazon Redshift',
  'Amazon Relational Database Service',
  'Amazon Route 53',
  'Amazon Simple Notification Service',
  'Amazon Simple Queue Service',
  'Amazon Simple Storage Service',
  'AmazonCloudWatch',
  'AWS CloudTrail',
  'AWS Data Transfer',
  'AWS Database Migration Service',
  'AWS Key Management Service',
  'AWS Lambda',
  'AWS Secrets Manager',
  'AWS Support (Business)',
  'AWS WAF',
  'Trustwave Managed Rules for AWS WAF - ModSecurity Virtual Patching sold by TrustWave Holdings, Inc.'],
 'Acqcom Digital Marketing': ['Web Media Fee'],
 'AdLift': ['Content Marketing', 'SEO Services'],
 'Adjust': ['Additional attributions', '

# prepare training

In [6]:
train_data=pd.read_excel(Meta_file,sheetname='train')
eval_data=pd.read_excel(Meta_file,sheetname='eval')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


## preprocessing and merge columns

In [7]:
def prepare_dataset(dataframe):

    for col in dataframe.columns:
        dataframe[col]=dataframe[col].apply(str).apply(str.strip)

    sep_token=' '
    dataframe=dataframe.fillna('  ')
    dataframe=dataframe.astype(str)
    
    # create a column that merges line_item_name and line_item_description
    dataframe['data']=dataframe.line_item_name+sep_token+dataframe.line_item_description
    return dataframe

In [8]:
train_data=prepare_dataset(train_data)

eval_data=prepare_dataset(eval_data)

# Syntacial Similarity 

###  Notes 

<ul>
    <li> I run prediction for each vendor not for each row so that when we calcualte sim score we could use matrix operation for batch processing for efficency.</li>
    <li> I generate the Syntacial Similarity Score based on word level similarity and char level similarity. I use 'line_item_name' and the merged column 'data' for computing similarity core at both word level and char level. </li>
    <li> I use a coef vairable  to weigh 'line_item_name' and the merged column 'line_item_description' differently. I weight 'line_item_name' more important than 'line_item_description' after inspecting the data closely</li>
</ul> 


In [9]:

def generate_similarity_score(rows,target_label,colname, analyzer='char_wb', ngram_range=(1,4), binary=True, embeding_mapping=None):
        coef=coef_map[colname]
        input_textual_data=rows[colname]
        
        if  embeding_mapping is not None:        
            input_vector=np.array([embeding_mapping[idx]  for idx in row_index_list])
            target_vector=np.array([label_embedding_map_dict[target_vendor][i]  for i in target_label])
        else:
            cv = CountVectorizer(analyzer='char_wb', ngram_range=ngram_range, vocabulary=None, binary=binary,min_df =0)
            target_vector=cv.fit_transform(target_label).toarray()
            input_vector=cv.transform(input_textual_data).toarray()
        
        cos_sim=cosine_similarity(input_vector,target_vector)
        sub_score=coef*cos_sim
        return sub_score


In [10]:
coef_map={'line_item_name':1, 
          'line_item_description':0.5,
          'data':1}

In [11]:
Syntac_result_df=pd.DataFrame()
for target_vendor in vendor_list:

    # retrieve all the assoiated canonical_line_item_name under each vendor
    target_label=np.array(label_map_dict[target_vendor])
    
    
    #get corresponding texutal data from training data
    
    matching_rows=train_data[train_data.canonical_vendor_name==target_vendor]
    
        
    if len(matching_rows)==0:
        continue
    
    class_map={label:i for i,label in enumerate(target_label)}

    inverse_class_map={i:label for i,label in enumerate(target_label)}

    # get  character based vocabs from the labels 

    score=np.zeros((len(matching_rows),len(target_label)))
    
    coef_sum=sum( i for i in coef_map.values())
    
    
    
    score+=generate_similarity_score(matching_rows,target_label,'line_item_name', analyzer='char_wb', ngram_range=(1,4), binary=True)
    
    score+=generate_similarity_score(matching_rows,target_label,'line_item_description', analyzer='char_wb', ngram_range=(1,4), binary=True)
    
   
    score+=generate_similarity_score(matching_rows,target_label,'data', analyzer='word', ngram_range=(1,2), binary=True)
    
    
    # averaging the cos_sim score matrix
    cos_matrix=score/(coef_sum)
    
    # find top k matching
    
    
    topk=3
    
    indices,scores=findknn(cos_matrix,topk)
    
    true_label_index= np.array([class_map[i]  for i in matching_rows.canonical_line_item_name])
    
    # get vector representation
    temp_result_df=train_data[train_data.canonical_vendor_name==target_vendor].copy()
    
   
    for i in range(indices.shape[1]):
        indice_mapping=target_label[indices[:,i]]
        indice_scores=scores[:,i]
    
        temp_result_df['syntatical_prediction_{}'.format(i+1)]=indice_mapping

        temp_result_df['syntatical_score_{}'.format(i+1)]=indice_scores
    
    Syntac_result_df=pd.concat([Syntac_result_df,temp_result_df],axis=0)

    acc=np.sum(true_label_index==indices[0].reshape(-1))/len(true_label_index)

    
Syntac_result_df=Syntac_result_df.reindex(np.arange(len(Syntac_result_df)))
Syntac_result_df=Syntac_result_df.drop('data',axis=1).reset_index().set_index(['canonical_vendor_name','index'])

Syntac_false_df= Syntac_result_df[Syntac_result_df.canonical_line_item_name!=Syntac_result_df['syntatical_prediction_1']]
print("overall acc: "  ,1-len(Syntac_false_df)/len(Syntac_result_df))

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:62: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


overall acc:  0.8710166919575114


In [12]:
Syntac_result_df.head()

canonical_line_item_name  \
canonical_vendor_name    index                            
10 Minute Ventures       0          Management Services   
Acqcom Digital Marketing 1                Web Media Fee   
                         2                Web Media Fee   
Adjust                   3             Business Package   
AdLift                   4                 SEO Services   

                                      line_item_description  \
canonical_vendor_name    index                                
10 Minute Ventures       0              April 2019 Services   
Acqcom Digital Marketing 1      ($249,300 x 12% Commission)   
                         2      ($180,000 x 12% Commission)   
Adjust                   3                              nan   
AdLift                   4                              nan   

                                     line_item_name syntatical_prediction_1  \
canonical_vendor_name    index                                                
10 Minute Ventures       0      Management Services     Management Services   
Acqcom Digital Marketing 1       June Web Media Fee           Web Media Fee   
                         2       June Web Media Fee           Web Media Fee   
Adjust                   3         Business Package        Business Package   
AdLift                   4             SEO Services            SEO Services   

                                syntatical_prediction_2  \
canonical_vendor_name    index                            
10 Minute Ventures       0                          NaN   
Acqcom Digital Marketing 1                          NaN   
                         2                          NaN   
Adjust                   3      Additional attributions   
AdLift                   4            Content Marketing   

                               syntatical_prediction_3  syntatical_score_1  \
canonical_vendor_name    index                                               
10 Minute Ventures       0                         NaN            0.939262   
Acqcom Digital Marketing 1                         NaN            0.853452   
                         2                         NaN            0.853452   
Adjust                   3                         NaN            0.833013   
AdLift                   4                         NaN            0.799208   

                                syntatical_score_2  syntatical_score_3  
canonical_vendor_name    index                                          
10 Minute Ventures       0                     NaN                 NaN  
Acqcom Digital Marketing 1                     NaN                 NaN  
                         2                     NaN                 NaN  
Adjust                   3                0.233160                 NaN  
AdLift                   4                0.225456                 NaN

## check those false predicted rows

In [13]:
Syntac_false_df

canonical_line_item_name  \
canonical_vendor_name index                                                      
Amazon Business       20           Anti-Theft Adjustable Tablet Security Stand   
Apple                 98                                            APPLECARE+   
Apple Search Ads      106                              Apple Search Ads: Other   
Athorus               111                                 Hourly Services: BYH   
Cleveland Scott York  139    UK Full Availability Trade Mark Register Searc...   
Cooper & Dunham       169                                 Hourly Services: LAA   
                      170                                 Hourly Services: LAA   
                      171                                 Hourly Services: LAA   
                      172                                 Hourly Services: LAA   
                      173                                 Hourly Services: LAA   
Freshworks            273                                  Estate Monthly plan   
GitHub                279                                          GitHub Plan   
Graphite Financial    307                               Hourly Services: Other   
                      308                               Hourly Services: Other   
                      309                               Hourly Services: Other   
                      310                            Hourly Services: Projects   
                      311                            Hourly Services: Projects   
                      312                            Hourly Services: Projects   
                      313                            Hourly Services: Projects   
                      314                            Hourly Services: Projects   
                      315                            Hourly Services: Projects   
                      316                            Hourly Services: Projects   
                      317                            Hourly Services: Projects   
                      318                            Hourly Services: Projects   
                      319                            Hourly Services: Projects   
                      320                            Hourly Services: Projects   
                      321                            Hourly Services: Projects   
                      322                            Hourly Services: Projects   
                      323                            Hourly Services: Projects   
                      324                            Hourly Services: Projects   
...                                                                        ...   
Perr&Knight           464                  Hourly Services: Accredited Actuary   
                      476                                      Travel Expenses   
Slack                 501                                 Fair billing credits   
STIGroup              518                                    Billable Expenses   
                      524                           Billable Time - Consultant   
                      531                       Non-Billable Time: Support T&M   
                      533    NYDFS Cybersecurity Program Implementation & O...   
Westmont Associates   569                                  Hourly Services: CC   
                      570                                  Hourly Services: CC   
                      576                                  Hourly Services: LM   
                      577                                  Hourly Services: LM   
                      579                                  Hourly Services: LM   
                      580                                  Hourly Services: LM   
                      581                                  Hourly Services: LM   
                      582                                  Hourly Services: LM   
                      583                                  Hourly Services: LM   
                      584           

## retrive the top_1 score and assign it as our synctaical scores 

In [14]:
#train_result_df1=result_df[['line_item_name','line_item_description','canonical_line_item_name','matched_item_name_top_1','sim_score_top_1']]

Syntac_result_df=Syntac_result_df.rename(columns={'canonical_line_item_name':'true_label'})

# Semantic Similarity

In [15]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
embed = hub.Module(module_url)


In [16]:
def get_embeding_from_USE(model,text):
    
    sentence_holder = tf.placeholder(tf.string, shape=(None))
    embedding_holder = model(sentence_holder)
    with tf.Session() as session:
        K.set_session(session)
        session.run(tf.global_variables_initializer())  
        session.run(tf.tables_initializer())
        input_embed = session.run(embedding_holder, feed_dict={sentence_holder: text})
        #input_embed2= session.run(embedding_holder, feed_dict={sentence_holder: text2})
    return input_embed


In [17]:
train_data.data.to_list()

['Management Services April 2019 Services',
 'June Web Media Fee ($249,300 x 12% Commission)',
 'June Web Media Fee ($180,000 x 12% Commission)',
 'Business Package nan',
 'SEO Services nan',
 'Creative Cloud All Apps nan',
 'YouTube Integration - 50% Final nan',
 '[100 Pack] 10 oz. White Paper Hot Cups - Coffee Cups nan',
 '1 Kitchen Sink Strainer Basket Catcher (2-pack) - 4.5" Diameter, Wide Rim Perfect for Most Sink Drains, Anti-Clogging Micro-Perforation 2mm Holes, Rust nan',
 '1 Prime-Line U 9940 Mailbox Lock - Replacement, Multipurpose Mailbox Lock for Several Brands - Brass Finish, ILCO 1003M Keyway, Opens Counter-Clockwise B00PJ95PSO',
 '100 PCS Binder Clips + 200PCS Bonus Paper Clips -6 Assorted Size Paper Clamps Clips with 28mm 32mm Paper Clips (300) nan',
 '18 oz Party Cups, 96 Count - Black, Purple, Pumpkin Orange - 32 Each Color nan',
 '4 Oz. White Paper Hot Coffee Cup For Espresso, Nespresso, Lavazza, Sampling Cup 100 Pack nan',
 '8 AmazonBasics USB Type-C to USB 3.1 Gen1

In [18]:
text=train_data.data[:4].to_list()

In [19]:
label_input=meta_df.canonical_line_item_name.to_list()

## get embedding for all the canonical_line_item_name


In [20]:
label_embedding=get_embeding_from_USE(embed,label_list)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [21]:

label_embedding_map_dict=defaultdict(dict)
for i in range(len(meta_df)):
    row=meta_df.iloc[i]
    label_embedding_map_dict[row.canonical_vendor_name][row.canonical_line_item_name]=label_embedding[i]

## get embedding for line item name + description in trainset

In [22]:
train_input=train_data.data.to_list()
train_embedding=get_embeding_from_USE(embed,train_input)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## map embedding to correct row index

In [23]:

def prepare_embedding_mapping(df, col):
    input_=df[col].to_list()
    embedding=get_embeding_from_USE(embed,input_)
   
    embedding_map_dict=defaultdict()
    for i,idx in enumerate(df.index):
        embedding_map_dict[idx]=embedding[i]
    return embedding_map_dict

    

In [24]:
embedding1=prepare_embedding_mapping(train_data, col='line_item_name')

embedding2=prepare_embedding_mapping(train_data, col='line_item_description')

embedding3=prepare_embedding_mapping(train_data, col='data')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [25]:
coef_map={'line_item_name':1, 
          'line_item_description':0.5,
          'data':1}

Semantic_result_df=pd.DataFrame()
for target_vendor in vendor_list:
    
    # retrieve all the assoiated canonical_line_item_name under each vendor
    target_label=np.array(label_map_dict[target_vendor])
    
    
    #get corresponding texutal data from training data
    
    
    matching_rows=train_data[train_data.canonical_vendor_name==target_vendor]
    
        
    if len(matching_rows)==0:
        continue
 
    class_map={label:i for i,label in enumerate(target_label)}

    inverse_class_map={i:label for i,label in enumerate(target_label)}
    # get  character based vocabs from the labels 

    score=np.zeros((len(matching_rows),len(target_label)))
  
    coef_sum=sum( i for i in coef_map.values())
    row_index_list=matching_rows.index

    
    score+=generate_similarity_score(matching_rows,target_label,colname='line_item_name', embeding_mapping=embedding1)
    score+=generate_similarity_score(matching_rows,target_label,colname='line_item_description', embeding_mapping=embedding2)
    score+=generate_similarity_score(matching_rows,target_label,colname='data', embeding_mapping=embedding3)

    cos_matrix=score/(coef_sum)
    
    # find top one matching
    topk=3
    indices,scores=findknn(cos_matrix,topk)
    

    true_label_index= np.array([class_map[i]  for i in matching_rows.canonical_line_item_name])
    


    temp_result_df=train_data[train_data.canonical_vendor_name==target_vendor].copy()

    
    
    for i in range(indices.shape[1]):
        indice_mapping=target_label[indices[:,i]]
        indice_scores=scores[:,i]
    
        temp_result_df['semantic_prediction_{}'.format(i+1)]=indice_mapping

        temp_result_df['semantic_score_{}'.format(i+1)]=indice_scores
        
        
    
    Semantic_result_df=pd.concat([Semantic_result_df,temp_result_df],axis=0)

    acc=np.sum(true_label_index==indices[0].reshape(-1))/len(true_label_index)
    
    
Semantic_result_df=Semantic_result_df.reindex(np.arange(len(Semantic_result_df)))
Semantic_result_df=Semantic_result_df.drop('data',axis=1).reset_index().set_index(['canonical_vendor_name','index'])

Semantic_false_df= Semantic_result_df[Semantic_result_df.canonical_line_item_name!=Semantic_result_df['semantic_prediction_1']]
print("overall acc: "  ,1-len(Semantic_false_df)/len(Semantic_result_df))

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:63: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


overall acc:  0.8148710166919575


In [26]:
Semantic_false_df

canonical_line_item_name  \
canonical_vendor_name index                                                      
Amazon Business       20           Anti-Theft Adjustable Tablet Security Stand   
Amelia Willson        76                                     1,500-2,000 words   
                      77                                     1,500-2,000 words   
                      78                                     1,500-2,000 words   
                      83                                       900-1,500 words   
Andersen Tax          91                       Hourly Services: Legal Services   
                      92                       Hourly Services: Legal Services   
Apple                 97                                            APPLECARE+   
                      99                                            APPLECARE+   
Apple Search Ads      106                              Apple Search Ads: Other   
Athorus               110                                 Hourly Services: BYH   
Cleveland Scott York  139    UK Full Availability Trade Mark Register Searc...   
Cooper & Dunham       169                                 Hourly Services: LAA   
                      170                                 Hourly Services: LAA   
                      171                                 Hourly Services: LAA   
                      172                                 Hourly Services: LAA   
                      173                                 Hourly Services: LAA   
CSC                   192                                       FOREIGN FILING   
                      193                                       FOREIGN FILING   
                      194                                       FOREIGN FILING   
                      195                                       FOREIGN FILING   
                      202                                       FOREIGN FILING   
                      203                                       FOREIGN FILING   
                      204                                       FOREIGN FILING   
                      205                                       FOREIGN FILING   
                      206                                       FOREIGN FILING   
                      207                                       FOREIGN FILING   
                      210                                       FOREIGN FILING   
                      213                                       FOREIGN FILING   
                      216                                  NO STATE FILING FEE   
...                                                                        ...   
Westmont Associates   569                                  Hourly Services: CC   
                      570                                  Hourly Services: CC   
                      571                                 Hourly Services: DJR   
                      573                                 Hourly Services: JFO   
                      576                                  Hourly Services: LM   
                      577                                  Hourly Services: LM   
                      580                                  Hourly Services: LM   
                      582                                  Hourly Services: LM   
                      583                                  Hourly Services: LM   
                      584                                  Hourly Services: LM   
                      585                                  Hourly Services: LM   
                      589                                  Hourly Services: SR   
                      590                                  Hourly Services: SR   
                      591                             Non-Hourly Services: A&A   
                      592                             Non-Hourly Services: A&A   
                      593                             Non-Hourly Services: A&A   
                      594           

In [27]:
Semantic_result_df

canonical_line_item_name  \
canonical_vendor_name    index                                                      
10 Minute Ventures       0                                    Management Services   
Acqcom Digital Marketing 1                                          Web Media Fee   
                         2                                          Web Media Fee   
Adjust                   3                                       Business Package   
AdLift                   4                                           SEO Services   
Adobe                    5                                Creative Cloud All Apps   
Alex Gasaway             6                                    YouTube Integration   
Amazon Business          7      [100 Pack] 10 oz. White Paper Hot Cups - Coffe...   
                         8      1 Kitchen Sink Strainer Basket Catcher (2-pack...   
                         9      1 Prime-Line U 9940 Mailbox Lock - Replacement...   
                         10     100 PCS Binder Clips + 200PCS Bonus Paper Clip...   
                         11     18 oz Party Cups, 96 Count - Black, Purple, Pu...   
                         12     4 Oz. White Paper Hot Coffee Cup For Espresso,...   
                         13     8 AmazonBasics USB Type-C to USB 3.1 Gen1 Fema...   
                         14                  AboveTEK Sleek Magnetic Tablet Stand   
                         15     Achim Home Furnishings Cordless Honeycomb Cell...   
                         16     Aluf Plastics 55-60 Gallon Blue Trash Bags for...   
                         17     Amazon Brand - Solimo Facial Tissues with Loti...   
                         18     AmazonBasics File Folders with Reinforced Tab ...   
                         19     AmazonBasics Multipurpose Copy Printer Paper -...   
                         20           Anti-Theft Adjustable Tablet Security Stand   
                         21     Anti-Theft Adjustable Tablet Security Stand - ...   
                         22     Apple iPad 4 16GB 9.7in Retina Display WiFi Bl...   
                         23     Apple Magic Keyboard with Numeric Keypad (Wire...   
                         24     Apple Magic Mouse 2 (Wireless, Rechargable) - ...   
                         25     Apple Power Adapter Extension Cable (for MacBo...   
                         26     Apple Thunderbolt 3 (USB-C) to Thunderbolt 2 A...   
                         27                         Apple USB-C Charge Cable (2m)   
                         28     Bankers Box SmoothMove Classic Moving Kit Boxe...   
                         29                               Brother P-touch, PTD210   
...                                                                           ...   
Xiamen ZhiZhi Tech       629                                             iOS: Tab   
                         630                                             iOS: Tab   
                         631                                             iOS: Tab   
                         632                                       Misc. expenses   
                         633                                       Misc. expenses   
                         634                                       Misc. expenses   
                         635                                       Misc. expenses   
                         636                                       Misc. expenses   
                         637                                       Misc. expenses   
                         638                                       Misc. expenses   
                         639                                       Misc. expenses   
                         640                                       Misc. expenses   
                         641                                       Misc. expenses   
                         642                                       Misc. expenses   
                         643                                  

# evaluate

In [28]:
final_eval_df= pd.concat([Syntac_result_df,Semantic_result_df.drop(columns=['canonical_line_item_name','line_item_name','line_item_description'],axis=1)],axis=1)

## mistakes made by two models

In [29]:
final_eval_df[(final_eval_df.true_label!=final_eval_df.syntatical_prediction_1)&(final_eval_df.true_label==final_eval_df.semantic_prediction_1)]

true_label  \
canonical_vendor_name index                                                      
Apple                 98                                            APPLECARE+   
Athorus               111                                 Hourly Services: BYH   
Freshworks            273                                  Estate Monthly plan   
Graphite Financial    320                            Hourly Services: Projects   
                      325                            Hourly Services: Projects   
                      344                                       Unbilled Hours   
Joseph W Russo        372                                             Expenses   
                      373                                             Expenses   
Maddie Shepherd       392                                            Blog Post   
                      396                                            Blog Post   
Microsoft Azure       434    Storage: Premium SSD Managed Disks P10 Disks U...   
Perr&Knight           463                  Hourly Services: Accredited Actuary   
                      464                  Hourly Services: Accredited Actuary   
STIGroup              518                                    Billable Expenses   
                      524                           Billable Time - Consultant   
Westmont Associates   579                                  Hourly Services: LM   
                      581                                  Hourly Services: LM   
                      595                             Non-Hourly Services: A&A   
Xiamen ZhiZhi Tech    655                                            Web: Fred   

                                                         line_item_description  \
canonical_vendor_name index                                                      
Apple                 98                                             S6531LL/A   
Athorus               111    Preparation and filing of an international tra...   
Freshworks            273                                  Estate Monthly Plan   
Graphite Financial    320    PROJECT: Verifly; TASK: Dynamics GP Functionality   
                      325          PROJECT: Verifly; TASK: Backup Sheet Update   
                      344                    Unbilled Hours- System Transition   
Joseph W Russo        372                      Car from train station to hotel   
                      373                              train ticket-round trip   
Maddie Shepherd       392                                                  nan   
                      396                                                  nan   
Microsoft Azure       434          Premium SSD Managed Disks P30 Disks US East   
Perr&Knight           463                                   Accredited Actuary   
                      464                                   Accredited Actuary   
STIGroup              518                           Support T&M Computer Asset   
                      524                                           Consultant   
Westmont Associates   579    Work with J. Smith on Designated Producer onbo...   
                      581                   Producer onboarding correspondence   
                      595                                                  A&A   
Xiamen ZhiZhi Tech    655    1. Thimble Monthly Webapp 2. Customer Referral...   

                                                                line_item_name  \
canonical_vendor_name index                                                      
Apple                 98                       AC+ IPAD/IPAD AIR/IPAD Mini-PHX   
Athorus               111                                          164.0001-MP   
Freshworks            273                                            Freshdesk   
Graphite Financial    320                   Accounting:Core_Accounting_Service   
                      325                   Accounting:Core_Accounting_Service   
                      344                   Acco

## mistakes made by semantic model but not syntactical model

In [32]:
final_eval_df[(final_eval_df.true_label==final_eval_df.syntatical_prediction_1)&(final_eval_df.true_label!=final_eval_df.semantic_prediction_1)]

true_label  \
canonical_vendor_name index                                                      
Amelia Willson        76                                     1,500-2,000 words   
                      77                                     1,500-2,000 words   
                      78                                     1,500-2,000 words   
                      83                                       900-1,500 words   
Andersen Tax          91                       Hourly Services: Legal Services   
                      92                       Hourly Services: Legal Services   
Apple                 97                                            APPLECARE+   
                      99                                            APPLECARE+   
Athorus               110                                 Hourly Services: BYH   
CSC                   192                                       FOREIGN FILING   
                      193                                       FOREIGN FILING   
                      194                                       FOREIGN FILING   
                      195                                       FOREIGN FILING   
                      202                                       FOREIGN FILING   
                      203                                       FOREIGN FILING   
                      204                                       FOREIGN FILING   
                      205                                       FOREIGN FILING   
                      206                                       FOREIGN FILING   
                      207                                       FOREIGN FILING   
                      210                                       FOREIGN FILING   
                      213                                       FOREIGN FILING   
                      216                                  NO STATE FILING FEE   
                      217                                  NO STATE FILING FEE   
                      239                         SPECIAL ARRANGEMENT DISCOUNT   
                      247                         SPECIAL ARRANGEMENT DISCOUNT   
Graphite Financial    297                          Hourly Services: Accounting   
                      298                          Hourly Services: Accounting   
                      299                          Hourly Services: Accounting   
                      300                          Hourly Services: Accounting   
                      301                          Hourly Services: Accounting   
                      302                          Hourly Services: Accounting   
                      303                          Hourly Services: Accounting   
                      304                          Hourly Services: Accounting   
                      305                          Hourly Services: Accounting   
                      306                          Hourly Services: Accounting   
Intrequant            359                                             Retainer   
                      360                                             Retainer   
                      362                                             Retainer   
Maddie Shepherd       389                                            Blog Post   
Perr&Knight           472             Hourly Services: Product Design Director   
                      474    Hourly Services: Product Design Senior Consultant   
                      475    Hourly Services: Product Design Senior Consultant   
Pingdom               477                                     Pingdom Advanced   
Westmont Associates   551                                Expenses: Filing Fees   
                      568                                 Hourly Services: BRH   
                      571                                 Hourly Services: DJR   
                      573                                 Hourly Services: JFO   
                      590                         

## mistakes made by syntactical model but not semantic model

In [33]:
final_eval_df[(final_eval_df.true_label!=final_eval_df.syntatical_prediction_1)&(final_eval_df.true_label==final_eval_df.semantic_prediction_1)]

true_label  \
canonical_vendor_name index                                                      
Apple                 98                                            APPLECARE+   
Athorus               111                                 Hourly Services: BYH   
Freshworks            273                                  Estate Monthly plan   
Graphite Financial    320                            Hourly Services: Projects   
                      325                            Hourly Services: Projects   
                      344                                       Unbilled Hours   
Joseph W Russo        372                                             Expenses   
                      373                                             Expenses   
Maddie Shepherd       392                                            Blog Post   
                      396                                            Blog Post   
Microsoft Azure       434    Storage: Premium SSD Managed Disks P10 Disks U...   
Perr&Knight           463                  Hourly Services: Accredited Actuary   
                      464                  Hourly Services: Accredited Actuary   
STIGroup              518                                    Billable Expenses   
                      524                           Billable Time - Consultant   
Westmont Associates   579                                  Hourly Services: LM   
                      581                                  Hourly Services: LM   
                      595                             Non-Hourly Services: A&A   
Xiamen ZhiZhi Tech    655                                            Web: Fred   

                                                         line_item_description  \
canonical_vendor_name index                                                      
Apple                 98                                             S6531LL/A   
Athorus               111    Preparation and filing of an international tra...   
Freshworks            273                                  Estate Monthly Plan   
Graphite Financial    320    PROJECT: Verifly; TASK: Dynamics GP Functionality   
                      325          PROJECT: Verifly; TASK: Backup Sheet Update   
                      344                    Unbilled Hours- System Transition   
Joseph W Russo        372                      Car from train station to hotel   
                      373                              train ticket-round trip   
Maddie Shepherd       392                                                  nan   
                      396                                                  nan   
Microsoft Azure       434          Premium SSD Managed Disks P30 Disks US East   
Perr&Knight           463                                   Accredited Actuary   
                      464                                   Accredited Actuary   
STIGroup              518                           Support T&M Computer Asset   
                      524                                           Consultant   
Westmont Associates   579    Work with J. Smith on Designated Producer onbo...   
                      581                   Producer onboarding correspondence   
                      595                                                  A&A   
Xiamen ZhiZhi Tech    655    1. Thimble Monthly Webapp 2. Customer Referral...   

                                                                line_item_name  \
canonical_vendor_name index                                                      
Apple                 98                       AC+ IPAD/IPAD AIR/IPAD Mini-PHX   
Athorus               111                                          164.0001-MP   
Freshworks            273                                            Freshdesk   
Graphite Financial    320                   Accounting:Core_Accounting_Service   
                      325                   Accounting:Core_Accounting_Service   
                      344                   Acco

# Combine two scores

##  After looking at the data, the syntacical score should be more important than semantics, weighting 1.5:1

In [34]:
Combine_result_df=pd.DataFrame()

coef_map={'line_item_name':1, 
          'line_item_description':0.5,
          'data':1}
for target_vendor in vendor_list:

    # retrieve all the assoiated canonical_line_item_name under each vendor
    target_label=np.array(label_map_dict[target_vendor])
    
    
    #get corresponding texutal data from training data
    
    matching_rows=train_data[train_data.canonical_vendor_name==target_vendor]
    
        
    if len(matching_rows)==0:
        continue
    
    class_map={label:i for i,label in enumerate(target_label)}

    inverse_class_map={i:label for i,label in enumerate(target_label)}

    # get  character based vocabs from the labels 

    score=np.zeros((len(matching_rows),len(target_label)))
    
    coef_sum=2.5*sum( i for i in coef_map.values())
    
    row_index_list=matching_rows.index
    
    score+=generate_similarity_score(matching_rows,target_label,'line_item_name', analyzer='char_wb', ngram_range=(1,4), binary=True)
    
    score+=generate_similarity_score(matching_rows,target_label,'line_item_description', analyzer='char_wb', ngram_range=(1,4), binary=True)
    
   
    score+=generate_similarity_score(matching_rows,target_label,'data', analyzer='word', ngram_range=(1,2), binary=True)
    
    
    
    score+=1.5*generate_similarity_score(matching_rows,target_label,colname='line_item_name', embeding_mapping=embedding1)
    score+=1.5*generate_similarity_score(matching_rows,target_label,colname='line_item_description', embeding_mapping=embedding2)
    score+=1.5*generate_similarity_score(matching_rows,target_label,colname='data', embeding_mapping=embedding3)
    
    
    # averaging the cos_sim score matrix
    cos_matrix=score/(coef_sum)
    
    # find top k matching
    
    
    topk=3
    
    indices,scores=findknn(cos_matrix,topk)
    
    true_label_index= np.array([class_map[i]  for i in matching_rows.canonical_line_item_name])
    
    # get vector representation
    temp_result_df=train_data[train_data.canonical_vendor_name==target_vendor].copy()
    
   
    for i in range(indices.shape[1]):
        indice_mapping=target_label[indices[:,i]]
        indice_scores=scores[:,i]
    
        temp_result_df['prediction_{}'.format(i+1)]=indice_mapping

        temp_result_df['score_{}'.format(i+1)]=indice_scores
    
    Combine_result_df=pd.concat([Combine_result_df,temp_result_df],axis=0)

    acc=np.sum(true_label_index==indices[0].reshape(-1))/len(true_label_index)

    
Combine_result_df=Combine_result_df.reindex(np.arange(len(Combine_result_df)))
Combine_result_df=Combine_result_df.drop('data',axis=1).reset_index().set_index(['canonical_vendor_name','index'])

Combine_false_df= Combine_result_df[Combine_result_df.canonical_line_item_name!=Combine_result_df['prediction_1']]
print("overall acc: "  ,1-len(Combine_false_df)/len(Combine_result_df))

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:70: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:72: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


overall acc:  0.8877086494688923


In [35]:
Combine_false_df

canonical_line_item_name  \
canonical_vendor_name index                                                      
Amazon Business       20           Anti-Theft Adjustable Tablet Security Stand   
Apple Search Ads      106                              Apple Search Ads: Other   
Athorus               110                                 Hourly Services: BYH   
                      111                                 Hourly Services: BYH   
Cleveland Scott York  139    UK Full Availability Trade Mark Register Searc...   
Cooper & Dunham       169                                 Hourly Services: LAA   
                      170                                 Hourly Services: LAA   
                      171                                 Hourly Services: LAA   
                      172                                 Hourly Services: LAA   
                      173                                 Hourly Services: LAA   
CSC                   195                                       FOREIGN FILING   
GitHub                279                                          GitHub Plan   
Graphite Financial    300                          Hourly Services: Accounting   
                      301                          Hourly Services: Accounting   
                      307                               Hourly Services: Other   
                      308                               Hourly Services: Other   
                      309                               Hourly Services: Other   
                      311                            Hourly Services: Projects   
                      312                            Hourly Services: Projects   
                      313                            Hourly Services: Projects   
                      314                            Hourly Services: Projects   
                      315                            Hourly Services: Projects   
                      317                            Hourly Services: Projects   
                      318                            Hourly Services: Projects   
                      319                            Hourly Services: Projects   
                      322                            Hourly Services: Projects   
                      324                            Hourly Services: Projects   
                      336                               Hourly Services: Tasks   
                      337                               Hourly Services: Tasks   
                      338                               Hourly Services: Tasks   
...                                                                        ...   
Maddie Shepherd       397                                            Blog Post   
Microsoft Azure       426                       Storage: Files LRS Data Stored   
                      436    Storage: Premium SSD Managed Disks P10 Disks U...   
Perr&Knight           476                                      Travel Expenses   
Slack                 501                                 Fair billing credits   
STIGroup              531                       Non-Billable Time: Support T&M   
                      533    NYDFS Cybersecurity Program Implementation & O...   
Westmont Associates   569                                  Hourly Services: CC   
                      570                                  Hourly Services: CC   
                      576                                  Hourly Services: LM   
                      577                                  Hourly Services: LM   
                      579                                  Hourly Services: LM   
                      580                                  Hourly Services: LM   
                      581                                  Hourly Services: LM   
                      582                                  Hourly Services: LM   
                      583                                  Hourly Services: LM   
                      584           

# generaete test results

In [36]:
evalutiaondf=pd.DataFrame()
evalutiaondf.columns

Index([], dtype='object')

In [37]:
evalutiaondf=pd.DataFrame()

coef_map={'line_item_name':1, 
          'line_item_description':0.5,
          'data':1}
for target_vendor in vendor_list:

    # retrieve all the assoiated canonical_line_item_name under each vendor
    
    target_label=np.array(label_map_dict[target_vendor])
    
    
    #get corresponding texutal data from training data
    
    matching_rows=eval_data[eval_data.canonical_vendor_name==target_vendor]
    
  
    if len(matching_rows)==0:
        continue
    
    class_map={label:i for i,label in enumerate(target_label)}

    inverse_class_map={i:label for i,label in enumerate(target_label)}

    # get  character based vocabs from the labels 

    score=np.zeros((len(matching_rows),len(target_label)))
    
    coef_sum=2.5*sum( i for i in coef_map.values())
    
    row_index_list=matching_rows.index
    
    score+=1.5*generate_similarity_score(matching_rows,target_label,'line_item_name', analyzer='char_wb', ngram_range=(1,4), binary=True)
    
    score+=1.5*generate_similarity_score(matching_rows,target_label,'line_item_description', analyzer='char_wb', ngram_range=(1,4), binary=True)
    
   
    score+=1.5*generate_similarity_score(matching_rows,target_label,'data', analyzer='word', ngram_range=(1,2), binary=True)
    
    
    
    score+=1*generate_similarity_score(matching_rows,target_label,colname='line_item_name', embeding_mapping=embedding1)
    score+=1*generate_similarity_score(matching_rows,target_label,colname='line_item_description', embeding_mapping=embedding2)
    score+=1*generate_similarity_score(matching_rows,target_label,colname='data', embeding_mapping=embedding3)
    
    
    # averaging the cos_sim score matrix
    cos_matrix=score/(coef_sum)
    
    # find top k matching
    
    
    topk=3
    
    indices,scores=findknn(cos_matrix,topk)
    
    

    true_label_index=None
    
    # get vector representation
    temp_result_df=eval_data[eval_data.canonical_vendor_name==target_vendor].copy()
    #print(temp_result_df.columns)
   
    for i in range(indices.shape[1]):
        indice_mapping=target_label[indices[:,i]]
        indice_scores=scores[:,i]
    
        temp_result_df['prediction_{}'.format(i+1)]=indice_mapping

        temp_result_df['score_{}'.format(i+1)]=indice_scores
    
    evalutiaondf=pd.concat([evalutiaondf,temp_result_df],axis=0)


/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:73: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [38]:
evalutiaondf

,canonical_line_item_name,canonical_vendor_name,data,line_item_description,line_item_name,prediction_1,prediction_2,prediction_3,score_1,score_2,score_3
0,nan,10 Minute Ventures,Management Services May 2019 Services,May 2019 Services,Management Services,Management Services,NaN,NaN,0.881744,NaN,NaN
55,nan,AWS,Amazon EC2 Container Registry (ECR) nan,nan,Amazon EC2 Container Registry (ECR),Amazon EC2 Container Registry (ECR),Amazon EC2 Container Service,Amazon Kinesis Firehose,0.619573,0.467535,0.358529
56,nan,AWS,AmazonCloudWatch nan,nan,AmazonCloudWatch,AmazonCloudWatch,Amazon CloudFront,Amazon Elastic Compute Cloud,0.555697,0.462843,0.432923
57,nan,AWS,Trustwave Managed Rules for AWS WAF - ModSecur...,nan,Trustwave Managed Rules for AWS WAF - ModSecur...,Trustwave Managed Rules for AWS WAF - ModSecur...,AWS Key Management Service,AWS Secrets Manager,0.595486,0.374351,0.350847
1,nan,Adobe,Acrobat Pro DC nan,nan,Acrobat Pro DC,Acrobat Pro DC,Creative Cloud All Apps,NaN,0.654661,0.256731,NaN
2,nan,Amazon Business,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...,nan,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...,SquareTrade B2B 3-Year Tablets Accidental Prot...,Black Vinyl Numbers Stickers - 6 Inch Self Adh...,0.596735,0.331324,0.326012
3,nan,Amazon Business,AmazonBasics AAA 1.5 Volt Performance Alkaline...,nan,AmazonBasics AAA 1.5 Volt Performance Alkaline...,AmazonBasics AAA 1.5 Volt Performance Alkaline...,AmazonBasics Mesh Trash Can Waste Basket,AmazonBasics Multipurpose Copy Printer Paper -...,0.548832,0.355390,0.345344
4,nan,Amazon Business,AmazonBasics Mesh Trash Can Waste Basket 1,1,AmazonBasics Mesh Trash Can Waste Basket,AmazonBasics Mesh Trash Can Waste Basket,AmazonBasics File Folders with Reinforced Tab ...,OliviaTree 5PCS Innovative Dish Washing Net Cl...,0.532401,0.331458,0.309608
5,nan,Amazon Business,AmazonFresh Mediterranean Extra Virgin Olive O...,B01N3LCEDL,AmazonFresh Mediterranean Extra Virgin Olive O...,AmazonFresh Mediterranean Extra Virgin Olive O...,AmazonBasics File Folders with Reinforced Tab ...,Wyze Cam Pan 1080p Pan/Tilt/Zoom Wi-Fi Indoor ...,0.544917,0.355018,0.353313
6,nan,Amazon Business,Apple 87W USB-C Power Adapter (for MacBook Pro...,nan,Apple 87W USB-C Power Adapter (for MacBook Pro),Apple 87W USB-C Power Adapter (for MacBook Pro),Apple Power Adapter Extension Cable (for MacBo...,Runpower 87W USB C Power Adapter Charger for M...,0.572617,0.429889,0.412783


# save result

In [39]:
eval_data_new=pd.read_excel(Meta_file,sheetname='eval')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [40]:
eval_data_new.canonical_line_item_name=evalutiaondf.prediction_1

In [41]:
eval_data_new.to_csv("prediction.csv", index=False,header=True)

In [42]:

result=pd.read_csv("prediction.csv")



In [43]:
result

,line_item_name,line_item_description,canonical_vendor_name,canonical_line_item_name
0,Management Services,May 2019 Services,10 Minute Ventures,Management Services
1,Acrobat Pro DC,NaN,Adobe,Acrobat Pro DC
2,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...,NaN,Amazon Business,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...
3,AmazonBasics AAA 1.5 Volt Performance Alkaline...,NaN,Amazon Business,AmazonBasics AAA 1.5 Volt Performance Alkaline...
4,AmazonBasics Mesh Trash Can Waste Basket,1,Amazon Business,AmazonBasics Mesh Trash Can Waste Basket
5,AmazonFresh Mediterranean Extra Virgin Olive O...,B01N3LCEDL,Amazon Business,AmazonFresh Mediterranean Extra Virgin Olive O...
6,Apple 87W USB-C Power Adapter (for MacBook Pro),NaN,Amazon Business,Apple 87W USB-C Power Adapter (for MacBook Pro)
7,Apple iPad 2 MC979LL/A 2nd Generation Tablet (...,NaN,Amazon Business,[100 Pack] 10 oz. White Paper Hot Cups - Coffe...
8,Apple iPad with Retina Display,"MD513LL/A (16GB, Wi-Fi, White) 4th Generation ...",Amazon Business,Apple iPad with Retina Display MD511LL/A (32GB...
9,Apple iPad with Retina Display MD511LL/A (32GB...,NaN,Amazon Business,Apple iPad with Retina Display MD511LL/A (32GB...
